In [146]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None


# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [147]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

# Select scenario

In [148]:
# scenarios
se2024 = ('se2024', r'E:\Projects\utah_bike_demand_model_2024')
se2024_bb = ('se2024_bb', r'E:\Projects\utah_bike_demand_model_2024_BB')
se2024_planned = ('se2024_planned', r'E:\Projects\utah_bike_demand_model_2024_Planned')
se2024_bb_planned = ('se2024_bb_planned', r'E:\Projects\utah_bike_demand_model_2024_BB_Planned')
se2050 = ('se2050', r'E:\Projects\utah_bike_demand_model_2050')
se2050_bb = ('se2050_bb', r'E:\Projects\utah_bike_demand_model_2050_BB')
se2050_planned = ('se2050_planned', r'E:\Projects\utah_bike_demand_model_2050_Planned')
se2050_bb_planned = ('se2050_bb_planned', r'E:\Projects\utah_bike_demand_model_2050_BB_Planned')

In [149]:
scenario = se2024_bb_planned

# Create output directories

In [150]:
# create output gdb 
outputs = [f'{scenario[1]}\Post_Process_Bike_Model_Outputs\Outputs', f'{scenario[0]}.gdb']
if not os.path.exists(outputs[0]): os.makedirs(outputs[0])
gdb = os.path.join(outputs[0], outputs[1])
if not arcpy.Exists(gdb): arcpy.CreateFileGDB_management(outputs[0], outputs[1])

## Join Bike volume data to links

In [151]:
# read in links csv and shapefile
links = pd.read_csv(f'{scenario[1]}\\Convert_MM_Network\\Outputs\\links.csv')
links_shp = pd.DataFrame.spatial.from_featureclass(f'{scenario[1]}\\Convert_MM_Network\\Outputs\\bike_network.gdb\\links')

# read in bike volume
bike_volume = pd.read_csv(f"{scenario[1]}\\Model_Outputs\\bike_vol.csv")
print(bike_volume.shape)

# fill bike volume NAs with 0
bike_volume['bike_vol'] = bike_volume['bike_vol'].fillna(0)

# REMOVE THIS WHEN TABLE BUG IS FIXED
bike_volume.dropna(inplace=True)

# convert node ids to int
bike_volume['from_node'] = bike_volume['from_node'].astype(int)
bike_volume['to_node'] = bike_volume['to_node'].astype(int)

# Create key to use for joining to links
bike_volume['key'] = np.where(bike_volume['from_node'] < bike_volume['to_node'], 
                              bike_volume['from_node'].astype(str) + "_"+ bike_volume['to_node'].astype(str), 
                              bike_volume['to_node'].astype(str) + "_"+ bike_volume['from_node'].astype(str))

# Create directional keys
bike_volume['ft_key'] = bike_volume['from_node'].astype(str) + "_"+ bike_volume['to_node'].astype(str)
bike_volume['tf_key'] = bike_volume['to_node'].astype(str) + "_"+ bike_volume['from_node'].astype(str)

# summarize trips in each direction
ft_vol_sum = pd.DataFrame(bike_volume.groupby('ft_key')['bike_vol'].sum())
tf_vol_sum = pd.DataFrame(bike_volume.groupby('tf_key')['bike_vol'].sum())

ft_vol_sum.columns = ['ft_bvol']
tf_vol_sum.columns = ['tf_bvol']

# summarize trips in both directions
volume_sum = bike_volume.groupby('key', as_index=False)['bike_vol'].sum()
volume_sum.rename({'bike_vol':'total_bvol'}, axis=1, inplace=True)

#Create FTkey and TF key to use for joining to bike volumes
links['key'] = np.where(links['from_node'].astype(int) < links['to_node'].astype(int), 
                              links['from_node'].astype(str) + "_"+ links['to_node'].astype(str), 
                              links['to_node'].astype(str) + "_"+ links['from_node'].astype(str))

links[['from_node', 'to_node','key']].head(10)
links2 = links[['link_id', 'key']].copy()

# join the links with the bike volumes using the common keys
link_bike_vol = links2.merge(volume_sum, left_on='key', right_on='key', how='left')
link_bike_vol2 = link_bike_vol.merge(ft_vol_sum, left_on='key', right_on='ft_key', how='left')
link_bike_vol3 = link_bike_vol2.merge(tf_vol_sum, left_on='key', right_on='tf_key', how='left')

link_bike_vol3['link_id'] = link_bike_vol3['link_id'].astype('int64')
links4 = links_shp.merge(link_bike_vol3, left_on='link_id', right_on='link_id', how='left')
links4['miles_trv'] = links4['Length_Miles'] * links4['total_bvol']
links4['miles_trv'] = links4['miles_trv'].round(2)
links4 = fill_na_sedf(links4)
links4.spatial.to_featureclass(location=os.path.join(gdb,"links_bike_volume"))

c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


(614362, 3)


'E:\\Projects\\utah_bike_demand_model_2024_BB_Planned\\Post_Process_Bike_Model_Outputs\\Outputs\\se2024_bb_planned.gdb\\links_bike_volume'

## Summarize zone trips by Attracting/Producing Zone

In [152]:
# read in zones
zones_shape = f'{scenario[1]}\\Create_Microzones\\Outputs\\microzones.gdb\\microzones'
if os.path.exists(zones_shape) == True:
   zones_sdf = pd.DataFrame.spatial.from_featureclass(zones_shape) 

else:
    zone_data = pd.read_csv(f'{scenario[1]}\\Create_Microzones\\Outputs\\microzones.csv').astype('Float64')
    zones_sdf =  pd.DataFrame.spatial.from_featureclass(r"E:\\Tasks\MICROZONES\\Inputs\\Microzones_20240815.shp")
    zones_sdf = zones_sdf[['SHAPE','zone_id']].merge(zone_data, on='zone_id', how='left')
    zones_sdf.spatial.to_featureclass(location=zones_shape,sanitize_columns=False)

### Read in trip tables, summarize, and format

In [153]:
def summarize_zones(trips_df, name):
    
    # summarize trips by attraction or production
    trips_sum_attr = pd.DataFrame(trips_df.groupby('azone')['trips'].sum())
    trips_sum_prod = pd.DataFrame(trips_df.groupby('pzone')['trips'].sum())
    
    # format tables
    trips_sum_attr['zone_id'] = trips_sum_attr.index
    trips_sum_attr.columns = [name + '_abk', 'zone_id']
    trips_sum_prod['zone_id'] = trips_sum_prod.index
    trips_sum_prod.columns = [name + '_pbk', 'zone_id']
    
    # join the attraction and production summary tables using zone id
    merged = trips_sum_attr.merge(trips_sum_prod, left_on='zone_id', right_on='zone_id', how='outer')
    return merged
    

In [154]:
# Discretionary trips (social trips, some recreation)
disc = pd.read_csv(os.path.join(scenario[1], "Model_Outputs", "disc_trip.parquet"))
disc_sum = summarize_zones(disc, 'disc')
del disc

# Maintenance trips (e.g. groceries)
maint = pd.read_csv(os.path.join(scenario[1], "Model_Outputs", "maint_trip.parquet"))
maint_sum = summarize_zones(maint, 'mnt')
del maint

# Maintenance trips non-home-based (e.g. groceries)
maint_nhb = pd.read_csv(os.path.join(scenario[1], "Model_Outputs", "maint_nhb_trip.parquet"))
maint_nhb_sum = summarize_zones(maint_nhb, 'mntnhb')
del maint_nhb

# Recreational family trips
rec_fam = pd.read_csv(os.path.join(scenario[1], "Model_Outputs", "rec_fam_trip.parquet"))
rec_fam_sum = summarize_zones(rec_fam, 'recfam')
del rec_fam

# Recreation long trips
rec_long = pd.read_csv(os.path.join(scenario[1], "Model_Outputs", "rec_long_trip.parquet"))
rec_long_sum = summarize_zones(rec_long, 'reclng')
del rec_long

# Recreation mountain bike trips
rec_mtb = pd.read_csv(os.path.join(scenario[1], "Model_Outputs", "rec_mtb_trip.parquet"))
rec_mtb_sum = summarize_zones(rec_mtb, 'recmtb')
del rec_mtb

# Recreation other trips (recreation that doesn't fall into family or long)
rec_oth = pd.read_csv(os.path.join(scenario[1], "Model_Outputs", "rec_oth_trip.parquet"))
rec_oth_sum = summarize_zones(rec_oth, 'recoth')
del rec_oth

# School (grade) trips
sch_grade = pd.read_csv(os.path.join(scenario[1], "Model_Outputs", "sch_grade_trip.parquet"))
sch_grade_sum = summarize_zones(sch_grade, 'grade')
del sch_grade

# School (university) trips
sch_univ = pd.read_csv(os.path.join(scenario[1], "Model_Outputs", "sch_univ_trip.parquet"))
sch_univ_sum = summarize_zones(sch_univ, 'univ')
del sch_univ

# Work trips
work = pd.read_csv(os.path.join(scenario[1], "Model_Outputs", "work_trip.parquet"))
work_sum = summarize_zones(work, 'wrk')
del work

# Work non-home-based trips
work_nhb = pd.read_csv(os.path.join(scenario[1], "Model_Outputs", "work_nhb_trip.parquet"))
work_nhb_sum = summarize_zones(work_nhb, 'wrknhb')
del work_nhb

## Merge trip summaries back to microzone shapefile

In [155]:
# Create a clean copy of zones dataset
zones2 = zones_sdf[['zone_id', 'area_sqmil', 'SHAPE']].copy()
zones2['zone_id'] = zones2['zone_id'].astype('int64')

# Join trip tables
zones2 = zones2.merge(disc_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(maint_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(maint_nhb_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_fam_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_long_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_mtb_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(rec_oth_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(sch_grade_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(sch_univ_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(work_sum, left_on='zone_id', right_on='zone_id', how='left')
zones2 = zones2.merge(work_nhb_sum, left_on='zone_id', right_on='zone_id', how='left')

In [156]:
# fill NAs where necessary
zones2 = fill_na_sedf(zones2)

# calc totals        
zones2['total_abk'] = (zones2['disc_abk'] + zones2['mnt_abk'] + zones2['mntnhb_abk'] + 
                       zones2['recfam_abk'] + zones2['reclng_abk'] + zones2['recmtb_abk'] + zones2['recoth_abk'] + 
                       zones2['grade_abk'] + zones2['univ_abk'] + zones2['wrk_abk'] + zones2['wrknhb_abk']) 

zones2['total_pbk'] = (zones2['disc_pbk'] + zones2['mnt_pbk'] + zones2['mntnhb_pbk'] + 
                       zones2['recfam_pbk'] + zones2['reclng_pbk'] + zones2['recmtb_pbk'] + zones2['recoth_pbk'] + 
                       zones2['grade_pbk'] + zones2['univ_pbk'] + zones2['wrk_pbk'] + zones2['wrknhb_pbk']) 

zones2.spatial.to_featureclass(location=os.path.join(gdb,"Microzone_Trip_Summaries"))

'E:\\Projects\\utah_bike_demand_model_2024_BB_Planned\\Post_Process_Bike_Model_Outputs\\Outputs\\se2024_bb_planned.gdb\\Microzone_Trip_Summaries'

## Merge zone attraction and production scores with the microzone geometry

In [157]:
# Create a clean copy of zones dataset
zones2 = zones_sdf[['zone_id', 'area_sqmil', 'SHAPE']].copy()
zones2['zone_id'] = zones2['zone_id'].astype('int64')

# NOTE: need to add zone_id to empty field in output csv
ascore = pd.read_csv(os.path.join(scenario[1], 'Model_Outputs','zone_attraction_size.csv'))
pscore = pd.read_csv(os.path.join(scenario[1], 'Model_Outputs','zone_production_size.csv'))

ascore = ascore.rename(columns={"Unnamed: 0": "zone_id"})
pscore = pscore.rename(columns={"Unnamed: 0": "zone_id"})

zones3a = zones2.merge(ascore, left_on='zone_id', right_on='zone_id', how='left')
zones3p = zones2.merge(pscore, left_on='zone_id', right_on='zone_id', how='left')

# fill NAs where necessary
zones3a = fill_na_sedf(zones3a)
zones3p = fill_na_sedf(zones3p)

zones3p.rename({'sch_grade_nhb': 'grade_nhb', 'sch_univ_nhb': 'univ_nhb', 'rec_oth_nhb':'recothnhb'}, axis=1, inplace=True)        
        
# Fill NAs with -1, then export to shape
zones3a.spatial.to_featureclass(location=os.path.join(gdb,"Microzone_A_Scores"))
zones3p.spatial.to_featureclass(location=os.path.join(gdb,"Microzone_P_Scores"))

'E:\\Projects\\utah_bike_demand_model_2024_BB_Planned\\Post_Process_Bike_Model_Outputs\\Outputs\\se2024_bb_planned.gdb\\Microzone_P_Scores'

## Get Centroid Nodes

In [158]:
nodes = pd.DataFrame.spatial.from_featureclass(os.path.join(scenario[1], 'Convert_MM_Network', 'Outputs', 'nodes.shp'))
nodes['node_id'] = nodes.index
nodes2 = nodes[['node_id', 'xcoord', 'ycoord', 'zcoord', 'SHAPE']].copy()
zones_sdf.rename({'NODE_ID':'node_id'}, axis=1, inplace=True)
centroids = nodes2.merge(zones_sdf[['node_id', 'zone_id']], on='node_id', how='inner')
centroids = centroids[['node_id', 'xcoord', 'ycoord', 'zcoord', 'zone_id', 'SHAPE']].copy()
centroids.spatial.to_featureclass(location=os.path.join(gdb,"Microzone_Centroids"))

'E:\\Projects\\utah_bike_demand_model_2024_BB_Planned\\Post_Process_Bike_Model_Outputs\\Outputs\\se2024_bb_planned.gdb\\Microzone_Centroids'